In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('vnexpress_articles.csv')
df.head()

,title,link,img,info,info_detail,page,category
0,Thailand eyes sending more workers to Singapore,https://e.vnexpress.net/news/business/economy/...,https://i1-english.vnecdn.net/2024/11/04/AAFP2...,Thailand’s Ministry of Labor is planning to ex...,Singapore is looking for welders in the shipbu...,https://e.vnexpress.net/,economy
1,How will Trump and Harris’s policies affect Vi...,https://e.vnexpress.net/news/business/economy/...,https://i1-english.vnecdn.net/2024/11/04/Trump...,Vietnam’s economy is likely to have similar im...,"The outcome of the presidential elections, set...",https://e.vnexpress.net/,economy
2,"Agriculture exports soar 20%, led by durian, c...",https://e.vnexpress.net/news/business/economy/...,https://i1-english.vnecdn.net/2024/11/02/ADSC0...,Vietnam’s agriculture exports in the first 10 ...,Durian exportscontributed $3 billion out of th...,https://e.vnexpress.net/,economy
3,Vietnamese guest workers earn highest incomes ...,https://e.vnexpress.net/news/business/economy/...,https://i1-english.vnecdn.net/2024/11/04/l1080...,The monthly incomes of Vietnamese guest worker...,According to the Vietnam Migration Profile 202...,https://e.vnexpress.net/,economy
4,Fresh coconuts quenching new overseas markets,https://e.vnexpress.net/news/business/economy/...,https://i1-english.vnecdn.net/2024/11/03/dua17...,Vietnam's fresh coconuts are being increasingl...,Vietnam holds the 6th position in the global c...,https://e.vnexpress.net/,economy


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5100 entries, 0 to 5099
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        5100 non-null   object
 1   link         5100 non-null   object
 2   img          5074 non-null   object
 3   info         4703 non-null   object
 4   info_detail  4558 non-null   object
 5   page         5100 non-null   object
 6   category     5100 non-null   object
dtypes: object(7)
memory usage: 279.0+ KB


In [4]:
# Gộp hai cột "info" và "info_detail"
df['combined'] = df['info'] + " " + df['info_detail']

Khi thực hiện crawl data, ta nhận thấy rằng nội dung đoạn đầu bài báo được sử dụng 2 thẻ khác nhau nên ta thực hiện crawl thành Cột 'info' và 'info_detail'. Bây giờ để có nội dung hoàn chỉnh, thực hiện kết hợp 2 cột lại với nhau

In [5]:
df = df.drop(['link','img', 'page', 'title','info','info_detail'], axis=1)
df

,category,combined
0,economy,Thailand’s Ministry of Labor is planning to ex...
1,economy,Vietnam’s economy is likely to have similar im...
2,economy,Vietnam’s agriculture exports in the first 10 ...
3,economy,The monthly incomes of Vietnamese guest worker...
4,economy,Vietnam's fresh coconuts are being increasingl...
...,...,...
5095,Education,"As the high school entrance exams approach, se..."
5096,Education,A schoolgirl in the northern Bac Ninh Province...
5097,Education,Two HCMC students have secured internships at ...
5098,Education,Hanoi high school student Vu Quoc Trung has ma...


In [6]:
df.isnull().sum()


category      0
combined    560
dtype: int64

In [7]:
# Xóa các hàng có giá trị rỗng trong cột 'combined'
df = df.dropna(subset=['combined'])

# Kiểm tra dữ liệu sau khi xóa
print("Dữ liệu sau khi xóa hàng rỗng:")
print(df.info())

Dữ liệu sau khi xóa hàng rỗng:
<class 'pandas.core.frame.DataFrame'>
Index: 4540 entries, 0 to 5099
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   category  4540 non-null   object
 1   combined  4540 non-null   object
dtypes: object(2)
memory usage: 106.4+ KB
None


In [8]:
# Đếm số lượng hàng cho từng giá trị trong cột 'category'
value_counts = df['category'].value_counts()
value_counts

category
markets         506
crime           505
data-speaks     502
Education       493
property        482
economy         467
traffic         428
environment     420
places          370
Food-recipes    367
Name: count, dtype: int64

PREPROCESS TEXT

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import re
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
import nltk

In [10]:
# Tải bộ stopwords (chỉ cần chạy lần đầu)
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HUUHUY\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [11]:
# --- 1. Tiền xử lý văn bản ---
def preprocess_text(text):
    if pd.isnull(text):
        return ""
    # Loại bỏ ký tự đặc biệt, số, và chuyển thành chữ thường
    text = re.sub(r"[^a-zA-Z]", " ", text).lower()
    # Tokenization và loại bỏ stop words
    words = text.split()
    stop_words = set(stopwords.words("english"))
    words = [word for word in words if word not in stop_words]
    # Stemming
    stemmer = SnowballStemmer("english")
    words = [stemmer.stem(word) for word in words]
    return " ".join(words)

In [12]:
# Tiền xử lý cột 'combined'
df['combined_cleaned'] = df['combined'].apply(preprocess_text)
df

C:\Users\HUUHUY\AppData\Local\Temp\ipykernel_15184\2322700629.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['combined_cleaned'] = df['combined'].apply(preprocess_text)


,category,combined,combined_cleaned
0,economy,Thailand’s Ministry of Labor is planning to ex...,thailand ministri labor plan export worker par...
1,economy,Vietnam’s economy is likely to have similar im...,vietnam economi like similar impact whether do...
2,economy,Vietnam’s agriculture exports in the first 10 ...,vietnam agricultur export first month surg yea...
3,economy,The monthly incomes of Vietnamese guest worker...,month incom vietnames guest worker abroad last...
4,economy,Vietnam's fresh coconuts are being increasingl...,vietnam fresh coconut increas consum intern ma...
...,...,...,...
5095,Education,"As the high school entrance exams approach, se...",high school entranc exam approach senior stude...
5096,Education,A schoolgirl in the northern Bac Ninh Province...,schoolgirl northern bac ninh provinc diagnos s...
5097,Education,Two HCMC students have secured internships at ...,two hcmc student secur internship boe u aviat ...
5098,Education,Hanoi high school student Vu Quoc Trung has ma...,hanoi high school student vu quoc trung manag ...


Vector

In [13]:
# Vector hóa cột 'combined_cleaned' bằng TF-IDF
tfidf_vectorizer = TfidfVectorizer(max_features=5000)  # Giới hạn 5000 từ phổ biến nhất
X_text = tfidf_vectorizer.fit_transform(df['combined_cleaned'])

# Encode cột 'category' bằng LabelEncoder
label_encoder = LabelEncoder()
y_labels = label_encoder.fit_transform(df['category'])

TF-IDF là một phương pháp phổ biến trong xử lý ngôn ngữ tự nhiên (NLP) để biểu diễn văn bản dưới dạng vector số, nhằm đo mức độ quan trọng của một từ trong một văn bản cụ thể so với toàn bộ tập dữ liệu (corpus).

Nếu một từ xuất hiện thường xuyên trong một tài liệu (TF cao) nhưng ít xuất hiện trong các tài liệu khác (IDF cao), thì TF-IDF sẽ cao. Điều này cho thấy từ đó có ý nghĩa quan trọng đối với tài liệu đó. Ngược lại, nếu từ xuất hiện ở hầu hết các tài liệu (IDF thấp), trọng số của nó sẽ thấp, ngay cả khi nó xuất hiện nhiều lần trong tài liệu.



In [14]:
# Chuyển ma trận TF-IDF thành DataFrame
tfidf_df = pd.DataFrame(X_text.toarray(), columns=tfidf_vectorizer.get_feature_names_out())

# Hiển thị 5 dòng đầu
print("Ma trận TF-IDF:")
print(tfidf_df.head())

Ma trận TF-IDF:
   abandon  abdul  abil  abl  abnorm  aboard    abroad  absenc  absolut  \
0      0.0    0.0   0.0  0.0     0.0     0.0  0.000000     0.0      0.0   
1      0.0    0.0   0.0  0.0     0.0     0.0  0.000000     0.0      0.0   
2      0.0    0.0   0.0  0.0     0.0     0.0  0.000000     0.0      0.0   
3      0.0    0.0   0.0  0.0     0.0     0.0  0.241057     0.0      0.0   
4      0.0    0.0   0.0  0.0     0.0     0.0  0.000000     0.0      0.0   

   absorb  ...  youth  youtub   yu  yuan  zalo   zealand  zero  zhang  zone  \
0     0.0  ...    0.0     0.0  0.0   0.0   0.0  0.000000   0.0    0.0   0.0   
1     0.0  ...    0.0     0.0  0.0   0.0   0.0  0.000000   0.0    0.0   0.0   
2     0.0  ...    0.0     0.0  0.0   0.0   0.0  0.000000   0.0    0.0   0.0   
3     0.0  ...    0.0     0.0  0.0   0.0   0.0  0.153813   0.0    0.0   0.0   
4     0.0  ...    0.0     0.0  0.0   0.0   0.0  0.000000   0.0    0.0   0.0   

   zoo  
0  0.0  
1  0.0  
2  0.0  
3  0.0  
4  0.0  

[5 

In [15]:
# Hiển thị nội dung
print("5 dòng đầu sau tiền xử lý và vector hóa:")
print(df[['combined_cleaned', 'category']].head())
print("\nHình dạng của ma trận TF-IDF:", X_text.shape)
print("\nDanh sách nhãn được mã hóa:")
print(pd.DataFrame({'category': label_encoder.classes_, 'encoded': range(len(label_encoder.classes_))}))

5 dòng đầu sau tiền xử lý và vector hóa:
                                    combined_cleaned category
0  thailand ministri labor plan export worker par...  economy
1  vietnam economi like similar impact whether do...  economy
2  vietnam agricultur export first month surg yea...  economy
3  month incom vietnames guest worker abroad last...  economy
4  vietnam fresh coconut increas consum intern ma...  economy

Hình dạng của ma trận TF-IDF: (4540, 5000)

Danh sách nhãn được mã hóa:
       category  encoded
0     Education        0
1  Food-recipes        1
2         crime        2
3   data-speaks        3
4       economy        4
5   environment        5
6       markets        6
7        places        7
8      property        8
9       traffic        9


In [16]:
# Lưu nhãn đã mã hóa
pd.DataFrame({'category': df['category'], 'encoded_category': y_labels}).to_csv("category_labels.csv", index=False)

In [17]:
tfidf_df.to_csv("combined_tfidf.csv", index=False)
print("Kết quả đã được lưu vào 'combined_tfidf.csv'")

Kết quả đã được lưu vào 'combined_tfidf.csv'
